# fdtd_python

> A Python package for Finite-Difference Time-Domain (FDTD) simulations.

In [109]:
#| export 
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from lmfit import Minimizer, Parameters, report_fit

In [110]:
#| default_exp fdtd_python

In [111]:
#| export
class fdtd_settings:
    def __init__(self,
                 simulation_size, # distance in microns (float)
                 simulation_time, # time in ps (#float)
                 thicknesses, # array
                 refractive_indices): # array
        
        self.simulation_size = simulation_size
        self.simulation_time = simulation_time
        self.thicknesses = thicknesses
        self.refractive_indices = refractive_indices

In [112]:
#| export
def easeInOutQuad(t_in):
    L = len(t_in)
    tout = np.zeros(L)
    t_final = t_in[-1]
    t_dummy = t_in/t_final
    for i,tval in enumerate(t_dummy):
        t = tval.copy()
        t*=2
        if t < 1:
            dummy = t * t / 2
        else:
            t-=1
            dummy = -((t-2)*t - 1) / 2
        tout[i] = dummy
    return tout*t_final

In [113]:
#| export 

def fit_refractive_index(l_mat, n_mat_exp, poles, repeat=10):
    """
    Fits refractive index data to a given amount of lorentzian susceptibility terms (sellmeier equations when no k). Returns a list of coefficients to be used in the fdtd_settings for that material
    Inputs
    l_mat: array of wavelengths in microns
    n_mat_exp: complex array of refractive indices
    poles: number of susceptibility terms to use (this will improve the accuracy of the fit but causes the computation time of fdtd through this material to scale linearly with the number of terms)
    repeat: how many times the fit is repeated with random initial values 
    
    Returns
    coeffs: list of coefficients for use in fdtd_settings
    """
    c = 300
    omega_mat = 2*np.pi*c/l_mat
    best_resid = 1e100

    for _ in range(repeat):
        params = Parameters()
        #params.add(f'eps_inf',value=10.0*np.random.random(),min=1.0)
        params.add(f'eps_inf',value=1,vary=False)

        # Adding parameters for lorentzian terms and fixing the imaginary part to 0 if there are no k values in the data
        for p in range(poles):
            params.add(f'eps_l_{p}',value=1e5*np.random.random(),min=0)
            params.add(f'omega_0_{p}',value=1e1*np.random.random())
            if np.all(np.isreal(n_mat_exp)):
                params.add(f'delta_{p}',value=0.0,vary=False)
            else:
                params.add(f'delta_{p}',value=10.0*np.random.random(),vary=True,min=0)
        # Defining the residual to minimize using the lorentzian terms and the refractive index from data
        def residual(params,omega_exp,n_exp):
            """Function to be minimised for fit"""
            poles = len(params.valuesdict())//3
            eps_inf = params[f'eps_inf']
            suscept = n_exp**2 - eps_inf
            model = 0
            for p in range(poles):
                eps_l = params[f'eps_l_{p}']
                omega_0 = params[f'omega_0_{p}']
                delta = params[f'delta_{p}']
                model = model + (eps_l*omega_0**2)/((omega_0**2) + 2j*delta*omega_exp - (omega_exp**2))
            resid = abs(model - suscept)**2
            return resid.view(float)
        minner = Minimizer(residual,params,fcn_args=(omega_mat,n_mat_exp))
        result_temp = minner.minimize(method='lbfgsb')
        resid = np.sum(abs(result_temp.residual)**2)
        # Selecting best residual
        if resid<best_resid:
            best_resid = resid
            print(best_resid)
            result = result_temp
    
    coeffs = result.params.valuesdict()
    coeffs = list(coeffs.values())
    # Convert to sellmeier like coeffs
    for idx in range((len(coeffs)-1)//3):
        temp = coeffs[3*idx+2]
        coeffs[3*idx+2] = 2*np.pi*c/coeffs[3*idx+2] # Convert omega_0 to C
        coeffs[3*idx+3] = 2*coeffs[3*idx+3]*coeffs[3*idx+2]/temp # Convert delta to D
    report_fit(result) # To print an lmfit fit report
    return coeffs


In [114]:
#| export 

def fdtd_run(wavelength, N_w, settings):
    """
    Calculates FDTD dispersion using a PLRC approach. 
    wavelength is in microns
    N_w: points per wavelength
    """
    c = 300 # microns/ps
    Courant = 1.00
    dx = wavelength/N_w # microns
    
    """
    """
    dt = dx*Courant/c
    t_final = settings.simulation_time # ps
    x_final = settings.simulation_size # microns

    t_arr = np.arange(0,t_final,dt)
    x_arr = np.arange(0,x_final,dx)


    tt = np.linspace(0,len(t_arr),len(t_arr))
    t_arr_eased = easeInOutQuad(tt)

    # fig, ax = plt.subplots()
    # dt = t_arr[1]-t_arr[0]
    # ax.plot([int(t) for t in t_arr_eased if (int(t) % 10)==0])
    
    t_size = len(t_arr)
    x_size = len(x_arr)
    
    imp0 = 377.0
    epsilon_0 = 1/(imp0*c)
    #LOSS = 0 # Ignored
    sigma = 0.0 # Frequency independent conductivity value through all materials

    cezCA = np.zeros((x_size))
    cezCB = np.zeros((x_size))
    cR1 = np.zeros((x_size,3),dtype=complex)
    cR2 = np.zeros((x_size,3),dtype=complex)
    cR3 = np.zeros((x_size,3),dtype=complex)
    

    buffer_size = (settings.simulation_size - np.sum(settings.thicknesses))/2
    left_edge = buffer_size
    right_edge = left_edge + np.sum(settings.thicknesses)

    lnew = np.zeros((len(settings.thicknesses),2))

    colors = ['blue','red','green']
    buffer_size = (settings.simulation_size - np.sum(settings.thicknesses))/2
    
    # Setting coefficient arrays
    for i,x in enumerate(x_arr):

        if 0<=x<left_edge or x>right_edge: # Free space
            cezCA[i] = epsilon_0/dt
            cezCB[i] = epsilon_0/dt
            
        for j,d in enumerate(settings.thicknesses): # Inside each layer
            left_edge_new = left_edge + np.sum(settings.thicknesses[:j])
            right_edge_new = left_edge + np.sum(settings.thicknesses[:j+1])          
            lnew[j,:] = [left_edge_new,right_edge_new]
            
            if left_edge_new<=x<=right_edge_new:
                 epsR = settings.refractive_indices[j] # Contains 6 values of sellmeier coefficients after index 0
                 
                 N = (len(epsR)-1)//3

                 B = np.zeros(N)
                 C = np.zeros(N)
                 D = np.zeros(N)
                 for idx in range(0,N):
                     B[idx] = epsR[3*idx+1]
                     C[idx] = epsR[3*idx+2]
                     D[idx] = epsR[3*idx+3]
                 
                 
                 epsilon_inf = epsR[0] + 1
                 # PLRC approach to coefficients

                #  print(B,C,D)
                 
                 alpha = np.zeros(N)
                 beta = np.zeros(N)
                 gamma = np.zeros(N)

                # Lorentzian susceptability coefficients
                 epsilon_l = np.zeros(N)
                 omega0 = np.zeros(N)
                 delta = np.zeros(N)

                 Ws = np.zeros(N,dtype=complex)
                 Qs = np.zeros(N,dtype=complex)
                 As = np.zeros(N,dtype=complex)
                 Bs = np.zeros(N,dtype=complex)
                 for n in range(len(epsilon_l)):
                    omega0[n] = 2*np.pi*c/C[n]
                    epsilon_l[n] = B[n]
                    delta[n] = (D[n]/C[n])*omega0[n]/2
                    
                    alpha[n] = delta[n]
                    beta[n] = np.sqrt(omega0[n]**2 - delta[n]**2)
                    gamma[n] = (epsilon_l[n]*omega0[n]**2)/beta[n]
   
                    Ws[n] = -1j*gamma[n]
                    Qs[n] = -alpha[n] + 1j*beta[n]
                    As[n] = -(Ws[n]/Qs[n]) - (Ws[n]/((Qs[n]**2)*dt))*(1 - np.exp(Qs[n]*dt))
                    Bs[n] = np.exp(Qs[n]*dt)*(Ws[n]/Qs[n]) + (Ws[n]/((Qs[n]**2)*dt))*(1 - np.exp(Qs[n]*dt))

                # Arrays actually used in the update equations:
                 cezCA[i] = (epsilon_0*epsilon_inf/dt) + (epsilon_0/dt)*np.sum(As.real) + sigma/2
                 cezCB[i] = (epsilon_0*epsilon_inf/dt) - (epsilon_0/dt)*np.sum(Bs.real) - sigma/2
                 cR1[i] = np.exp(Qs*dt)
                 cR2[i] = As
                 cR3[i] = Bs
                 



        #print(cfac[i])
    # print(alpha,beta,gamma)
    # print(f"Expected refractive index for wavelength {wavelength}um: ", (1 + np.sum(B/(1 - (C/wavelength)**2)))**0.5)
    #print(epsR)
    e_50 = np.zeros(t_size) # Efield at position 50

    fig, ax = plt.subplots()
    
    for i in range(np.size(lnew,0)):
        ax.fill_between((np.array(lnew[i,:])-buffer_size),y1=-1.2, y2=1.2,alpha=0.1,color=colors[i])

    #ax.set_ylim([-1.2,1.2])
    ax.set_xlabel(r'distance ($\mathrm{\mu}$m)')
    ax.set_ylabel('E(t) (norm.)')

    x_vals = [left_edge,right_edge]
    y_vals = [-1.2,1.2]
    artists = []
    offno = t_arr[-1]/4
    # print(offno, dx, c)
    # ax.vlines(x_vals,y_vals[0],y_vals[1],color='red',alpha=0.5)
    # ax.hlines(y=y_vals,xmin=left_edge,xmax=right_edge,color='red',alpha=0.5)

    # ax.fill_between(x=x_vals, y1=-1, y2=1,alpha=0.1,color='red')

    E_total = np.zeros((len(x_arr),len(t_arr)))
    
    # Time stepping

    ez_prev = np.zeros(x_size)
    ez = np.zeros(x_size)
    ez_next = np.zeros(x_size)

    
    # Recursive accumulator term

    R = np.zeros((x_size,3), dtype=complex)
    R_next = np.zeros((x_size,3), dtype=complex)
    Phi = np.zeros((x_size))
    
    hy = np.zeros(x_size)

    source_index = 10
    
    for i,t in enumerate(t_arr):
        
        hy[x_size-1] = hy[x_size-2]
        # Magnetic field update (non dispersive)
        for j in range(x_size-1):
            hy[j] = hy[j] + (ez[j+1] - ez[j])/imp0

        # Gaussian times sine wave source
        decay_time = 0.03 # Time for the pulse to reach 1/100th of its amplitude
        hy[source_index-1] -= np.exp(-(np.log(100)*((t - offno)/decay_time))**2)  * (np.sin(2*np.pi*c * (t - offno)/wavelength) / imp0 )

        # Continuous source
        #if (2*np.pi*c * (t - offno)/wavelength < 2*np.pi)   and ((t-offno)>0):
        #hy[source_index-1] -= (np.sin(2*np.pi*c * (t - offno)/wavelength) / imp0 )
        
        ez_next[0] = ez[1]

        # Electric field update dispersive
        for j in range(1,x_size):
            Phi[j] = (epsilon_0/dt)*np.sum((R[j]*(cR1[j]-1)).real)
            ez_next[j] = ((cezCB[j])/(cezCA[j]))*ez[j] + (1/cezCA[j])*((hy[j]-hy[j-1])/dx) - (1/cezCA[j])*Phi[j]
            if np.isnan(ez[j]) == True:
                print(f"NaN ELECTRIC FIELD VALUE AT j = {j}")
            
            
            

        # Gaussian times sine wave source
        ez_next[source_index] += np.exp(-(np.log(100)*((t - offno)/decay_time))**2) * (np.sin(2*np.pi*c * (t - offno)/wavelength))
        
        # Recursive accumulator update
        for j in range(1,x_size):
            R_next[j] = cR1[j]*R[j] + cR2[j]*ez_next[j] + cR3[j]*ez[j]


        ez_prev[:] = ez[:]
        ez[:] = ez_next[:]

        R[:] = R_next[:]
        
        E_total[:,i] = ez



        nos_frame = len(t_arr)//100

        if (i % nos_frame == 0):
            container = ax.plot(x_arr[10:]-buffer_size,ez[10:],color='black')
            #container = ax.plot(x_arr[10:] - buffer_size/wavelength,ez[10:],color='black')
            
            txt = ax.text(0.45,1.01,'{num:{fill}{width}} ps'.format(num=round(t_arr[i],1), fill='0', width=2),transform=ax.transAxes,fontsize="large")
        
            container.append(txt)
            # txt = plt.text(0.1,1,i)
            #container += ax2.plot(x_arr,h_snaps[:,i],color='green')
            artists.append(container)

    


    dpi = 100
    writer = animation.writers['pillow'](fps=30)
    #ani.save('test.mp4',writer=writer,dpi=dpi)
    
    ani = animation.ArtistAnimation(fig=fig, artists=artists)
    plt.show()
    ani.save(filename="pillow_example.gif", writer=writer, dpi=dpi)


    return x_arr[10:]-buffer_size,t_arr,E_total[10:,:]




In [115]:
#| export 

def fdtd_run_ade(wavelength, N_w, settings):
    """
    ADE implementation of dispersion
    !! ALMOST ALWAYS UNSTABLE !!
    wavelength is in microns
    N_w: points per wavelength
    """
    c = 300 # microns/ps
    Courant = 1.00
    dx = wavelength/N_w # microns
    
    """
    """
    dt = dx*Courant/c
    t_final = settings.simulation_time # ps
    x_final = settings.simulation_size # microns

    t_arr = np.arange(0,t_final,dt)
    x_arr = np.arange(0,x_final,dx)


    tt = np.linspace(0,len(t_arr),len(t_arr))
    t_arr_eased = easeInOutQuad(tt)

    # fig, ax = plt.subplots()
    # dt = t_arr[1]-t_arr[0]
    # ax.plot([int(t) for t in t_arr_eased if (int(t) % 10)==0])
    
    t_size = len(t_arr)
    x_size = len(x_arr)
    
    imp0 = 377.0
    epsilon_0 = 1/(imp0*c)
    mu_0 = imp0/c
    #LOSS = 0 # Ignored
    #epsR = np.zeros((x_size,7)) # Refractive indices
    cezPe = np.zeros((x_size))
    cezPm = np.zeros((x_size))
    cezChi = np.zeros((x_size))
    cezAlpha = np.zeros((x_size,3)) # Change later for better fits
    cezXi = np.zeros((x_size,3)) # Change later for better fits
    cezGamma = np.zeros((x_size,3)) # Change later for better fits
    
    #ADE_coeff = np.zeros((x_size,3,len(settings.refractive_indices[0][1::2]))) # Holds 3 coefficients for the additional ADE term for modelling dispersion with an ADE

    buffer_size = (settings.simulation_size - np.sum(settings.thicknesses))/2
    left_edge = buffer_size
    right_edge = left_edge + np.sum(settings.thicknesses)

    lnew = np.zeros((len(settings.thicknesses),2))

    colors = ['blue','red','green']
    buffer_size = (settings.simulation_size - np.sum(settings.thicknesses))/2
    
    # Setting coefficient arrays
    for i,x in enumerate(x_arr):

        if 0<=x<left_edge or x>right_edge: # Free space
            #epsR[i] = 1.0
            cezPe[i] = 1.0
            cezChi[i] = 0.0
            cezPm[i] = dt/epsilon_0
            
        for j,d in enumerate(settings.thicknesses): # Inside each layer
            left_edge_new = left_edge + np.sum(settings.thicknesses[:j])
            right_edge_new = left_edge + np.sum(settings.thicknesses[:j+1])          
            lnew[j,:] = [left_edge_new,right_edge_new]
            
            if left_edge_new<=x<=right_edge_new:
                 epsR = settings.refractive_indices[j] # Contains 6 values of sellmeier coefficients after index 0
                 
                 poles = (len(epsR)-1)//2
                 B = np.zeros(poles)
                 C = np.zeros(poles)
                 for idx in range(poles):
                     B[idx] = epsR[2*idx+1]
                     C[idx] = epsR[2*idx+2]
                 epsilon_inf = epsR[0] + 1.0
                 
                 # Lorentz oscillator model coefficients
                 D1 = np.zeros(poles)
                 D2 = np.zeros(poles)
                 D3 = np.zeros(poles)
                 
                 for p in range(len(B)):
                    omega0 = 2*np.pi*c/C[p]
                    epsilon_l = B[p]
                    delta = 1e-1*omega0
                    sigma = 0.0 # Conductivity
                    #print(omega0, epsilon_l, delta)

                    # ADE approach to coefficients
                    omegadt = omega0*dt
                    deltadt = delta*dt
                    D1[p] = (2 - omegadt**2)/(1 + deltadt)
                    D2[p] = (deltadt - 1)/(deltadt + 1)
                    D3[p] = (epsilon_0*epsilon_l*omegadt**2)#/(1 + deltadt)

                 D3_sum = np.sum(D3)/(4*dt)
                 nu = 1 + ((dt)/(epsilon_inf*epsilon_0))*(sigma/2 + D3_sum)
                 P_m = ((dt)/(epsilon_inf*epsilon_0))/nu
                 P_e = (1 - ((dt*sigma)/(2*epsilon_inf*epsilon_0)))/nu


                 E1 = (2*epsilon_0*epsilon_inf)/(2*epsilon_0*epsilon_inf + D3_sum/2)
                 E2 = (D3_sum/2)/(2*epsilon_0*epsilon_inf + D3_sum/2)
                 E3 = (2*dt)/(2*epsilon_0*epsilon_inf + D3_sum/2)

                 cezPe[i] = P_e
                 cezPm[i] = P_m
                 cezChi[i] = D3_sum
                 for p in range(len(B)):
                    cezAlpha[i][p] = D1[p]
                    cezXi[i][p] = D2[p]
                    cezGamma[i][p] = D3[p]


                        


        #print(cfac[i])
    #print(f"Expected refractive index for wavelength {wavelength}um: ", 1 + np.sum(B/(1 - (C/wavelength)**2)))
    #print(epsR)
    e_50 = np.zeros(t_size) # Efield at position 50

    fig, ax = plt.subplots()
    
    for i in range(np.size(lnew,0)):
        ax.fill_between((np.array(lnew[i,:])-buffer_size),y1=-1.2, y2=1.2,alpha=0.1,color=colors[i])

    #ax.set_ylim([-1.2,1.2])
    ax.set_xlabel(r'distance ($\mathrm{\mu}$m)')
    ax.set_ylabel('E(t) (norm.)')

    x_vals = [left_edge,right_edge]
    y_vals = [-1.2,1.2]
    artists = []
    offno = t_arr[-1]/4
    print(offno, dx, c)
    # ax.vlines(x_vals,y_vals[0],y_vals[1],color='red',alpha=0.5)
    # ax.hlines(y=y_vals,xmin=left_edge,xmax=right_edge,color='red',alpha=0.5)

    # ax.fill_between(x=x_vals, y1=-1, y2=1,alpha=0.1,color='red')

    E_total = np.zeros((len(x_arr),len(t_arr)))
    
    # Time stepping

    ez_prev = np.zeros(x_size)
    ez = np.zeros(x_size)
    ez_next = np.zeros(x_size)

    J_prev = np.zeros((x_size,3))
    J = np.zeros((x_size,3))
    J_next = np.zeros((x_size,3))
    #JJ = np.zeros(x_size)
    
    hy = np.zeros(x_size)

    source_index = 10
    for i,t in enumerate(t_arr):
        
        hy[x_size-1] = hy[x_size-2]
        # Magnetic field update (non dispersive)
        for j in range(x_size-1):
            hy[j] = hy[j] + (ez[j+1] - ez[j])/imp0

        # Gaussian times sine wave source
        decay_time = 0.03 # Time for the pulse to reach 1/100th of its amplitude
        hy[source_index-1] -= np.exp(-(np.log(100)*((t - offno)/decay_time))**2)  * (np.sin(2*np.pi*c * (t - offno)/wavelength) / imp0 )

        # Correction to make source one way
        #hy[source_index-1] -= ez[source_index]/imp0
        
        ez_next[0] = ez[1]

        # Electric field update dispersive
        for j in range(1,x_size):
            ez_next[j] = cezPe[j]*ez[j] + cezPm[j]*((hy[j]-hy[j-1])/(dx))
            ez_next[j] = ez_next[j] + cezPm[j]*cezChi[j]*ez_prev[j]
            for p in range(len(J[j])):
                ez_next[j] = ez_next[j] - (cezPm[j]*cezXi[j][p]/2)*J_prev[j][p] - (cezPm[j]*(cezAlpha[j][p]+1)/2)*J[j][p]
        
        # Gaussian times sine wave source
        ez_next[source_index] += np.exp(-(np.log(100)*((t - offno)/decay_time))**2) * (np.sin(2*np.pi*c * (t - offno)/wavelength))
        
        # Correction to make source one way
        #ez_next[source_index] -= cezh[j].real*hy[source_index-1]

        # Polarisation current update:
        for j in range(1,x_size):
            for p in range(len(J[j])):
                J_next[j][p] =  (cezGamma[j][p]*(ez_next[j] - ez_prev[j])/(2*dt)) + cezAlpha[j][p]*J[j][p] + cezXi[j][p]*J_prev[j][p] 
            if np.any(J[j]):
                print(J[j] , cezAlpha[j], cezXi[j], cezGamma[j], j)
        
        ez_prev[:] = ez[:]
        ez[:] = ez_next[:]

        J_prev[:] = J[:]
        J[:] = J_next[:]
        
        
        E_total[:,i] = ez


        nos_frame = len(t_arr)//100

        if (i % nos_frame == 0):
            container = ax.plot(x_arr[10:]-buffer_size,ez[10:],color='black')
            #container = ax.plot(x_arr[10:] - buffer_size/wavelength,ez[10:],color='black')
            
            txt = ax.text(0.45,1.01,'{num:{fill}{width}} ps'.format(num=round(t_arr[i],1), fill='0', width=2),transform=ax.transAxes,fontsize="large")
        
            container.append(txt)
            # txt = plt.text(0.1,1,i)
            #container += ax2.plot(x_arr,h_snaps[:,i],color='green')
            artists.append(container)

    


    dpi = 100
    writer = animation.writers['pillow'](fps=30)
    #ani.save('test.mp4',writer=writer,dpi=dpi)
    
    ani = animation.ArtistAnimation(fig=fig, artists=artists)
    plt.show()
    ani.save(filename="pillow_example_ade.gif", writer=writer, dpi=dpi)


    return x_arr[10:]-buffer_size,t_arr,E_total[10:,:]




In [116]:
#| export 

def fdtd_run_nd(wavelength, N_w, settings):
    """
    Non-dispersive fdtd that takes single values for refractive indices in its settings
    N_w: points per wavelength
    """
    c = 300 # microns/ps
    Courant = 1.0
    dx = wavelength/N_w # microns
    
    """
    """
    dt = dx*Courant/c
    t_final = settings.simulation_time # ps
    x_final = settings.simulation_size # microns

    t_arr = np.arange(0,t_final,dt)
    x_arr = np.arange(0,x_final,dx)


    tt = np.linspace(0,len(t_arr),len(t_arr))
    t_arr_eased = easeInOutQuad(tt)

    # fig, ax = plt.subplots()
    # dt = t_arr[1]-t_arr[0]
    # ax.plot([int(t) for t in t_arr_eased if (int(t) % 10)==0])
    
    t_size = len(t_arr)
    x_size = len(x_arr)
    
    imp0 = 377.0
    LOSS = 0.00 # is being ignored for now
    epsR = np.zeros(x_size)
    ceze = np.zeros(x_size) # Loss coefficient array for E
    cezh = np.zeros(x_size) # Loss coefficient array for H

    buffer_size = (settings.simulation_size - np.sum(settings.thicknesses))/2
    left_edge = buffer_size
    right_edge = left_edge + np.sum(settings.thicknesses)

    lnew = np.zeros((len(settings.thicknesses),2))

    colors = ['blue','red','green']
    buffer_size = (settings.simulation_size - np.sum(settings.thicknesses))/2
    
    # Setting coefficient arrays
    for i,x in enumerate(x_arr):

        if 0<=x<left_edge or x>right_edge: # Free space
            epsR[i] = 1.0
            ceze[i] = 1.0
            cezh[i] = imp0 * Courant
            
        for j,d in enumerate(settings.thicknesses): # Inside each layer
            left_edge_new = left_edge + np.sum(settings.thicknesses[:j])
            right_edge_new = left_edge + np.sum(settings.thicknesses[:j+1])          
            lnew[j,:] = [left_edge_new,right_edge_new]
            
            if left_edge_new<=x<=right_edge_new:
                 epsR[j] = settings.refractive_indices[j] 
                 ceze[i] = 1.0
                 cezh[i] = imp0 * Courant / epsR[j]
                 


            # print(epsR[i])
    #print(epsR)
    e_50 = np.zeros(t_size) # Efield at position 50

    fig, ax = plt.subplots()
    
    for i in range(np.size(lnew,0)):
        ax.fill_between((np.array(lnew[i,:])-buffer_size),y1=-1.2, y2=1.2,alpha=0.1,color=colors[i])

    #ax.set_ylim([-1.2,1.2])
    ax.set_xlabel(r'distance ($\mathrm{\mu}$m)')
    ax.set_ylabel('E(t) (norm.)')

    x_vals = [left_edge,right_edge]
    y_vals = [-1.2,1.2]
    artists = []
    offno = t_arr[-1]/4
    print(offno, dx, c)
    # ax.vlines(x_vals,y_vals[0],y_vals[1],color='red',alpha=0.5)
    # ax.hlines(y=y_vals,xmin=left_edge,xmax=right_edge,color='red',alpha=0.5)

    # ax.fill_between(x=x_vals, y1=-1, y2=1,alpha=0.1,color='red')

    E_total = np.zeros((len(x_arr),len(t_arr)))
    
    # Time stepping

    ez_prev = np.zeros(x_size)
    ez = np.zeros(x_size)
    ez_next = np.zeros(x_size)
    
    hy = np.zeros(x_size)

    source_index = 10#np.where(buffer_size/2 <= x_arr)[0][0]
    for i,t in enumerate(t_arr):
        
        hy[x_size-1] = hy[x_size-2]
        # Magnetic field update (non dispersive)
        for j in range(x_size-1):
            hy[j] = hy[j] + (ez[j+1] - ez[j])/imp0

        # Gaussian times sine wave source
        decay_time = 0.03 # Time for pulse to reach 1/100th of its amplitude
        hy[source_index-1] -= np.exp(-(np.log(100)*((t - offno)/decay_time))**2)  * (np.sin(2*np.pi*c * (t - offno)/wavelength) / imp0 )

        # One way correction
        #hy[source_index-1] -= ez[source_index]/imp0
        
        ez_next[0] = ez[1]

        # Electric field update dispersive
        for j in range(1,x_size): 
            ez_next[j] = ceze[j]*ez[j] + cezh[j]*(hy[j]-hy[j-1])

        # Gaussian times sine wave source
        ez_next[source_index] += np.exp(-(np.log(100)*((t - offno)/decay_time))**2) * (np.sin(2*np.pi*c * (t - offno)/wavelength))
        
        # One way correction
        #ez_next[source_index] -= cezh[j]*hy[source_index-1]
        #e_50[i] = ez_next[len(ez)-1]
        

        ez_prev[:] = ez[:]
        ez[:] = ez_next[:]
        
        E_total[:,i] = ez



        nos_frame = len(t_arr)//100

        if (i % nos_frame == 0):
            container = ax.plot(x_arr[10:]-buffer_size,ez[10:],color='black')
            #container = ax.plot(x_arr[10:] - buffer_size/wavelength,ez[10:],color='black')
            
            txt = ax.text(0.45,1.01,'{num:{fill}{width}} ps'.format(num=round(t_arr[i],1), fill='0', width=2),transform=ax.transAxes,fontsize="large")
        
            container.append(txt)
            # txt = plt.text(0.1,1,i)
            #container += ax2.plot(x_arr,h_snaps[:,i],color='green')
            artists.append(container)

    


    dpi = 100
    writer = animation.writers['pillow'](fps=30)
    #ani.save('test.mp4',writer=writer,dpi=dpi)
    
    ani = animation.ArtistAnimation(fig=fig, artists=artists)
    plt.show()
    ani.save(filename="pillow_example_nd.gif", writer=writer, dpi=dpi)



    return x_arr[10:]-buffer_size,t_arr,E_total[10:,:]

In [117]:
#| hide
import nbdev; nbdev.nbdev_export()